In [1]:
%matplotlib ipympl
def figure(name, nrows=1, ncols=1, *args, **kwargs):
    plt.close(name)
    return plt.subplots(nrows, ncols, num=name, *args, **kwargs)

import numpy as np
import pylab as plt
plt.style.use('default')

In [2]:
from scipy.integrate import solve_ivp
from ipywidgets import HBox, IntSlider, FloatSlider
from numpy import sin, cos

In [3]:
deg2rad = lambda deg: np.pi * deg / 180

def energy(y):
    T = 2*M*l**2*y[1]**2*sin(y[0])**2 + m*l**2*y[1]**2 + m*l**2*w**2*sin(y[0])**2
    U = -2*(m+M)*g*l*cos(y[0]) 
    return np.array((U, T))

In [4]:
# phi, dot(phi)
def derivative(t, y):
    dy = np.zeros(2)
    dy[0] = y[1]
    dy[1] = ((m*w**2 - 2*M*y[1]**2)*sin(y[0])*cos(y[0]) - g/l*(m+M)*sin(y[0]))/(2*M*sin(y[0])**2 + m)
    dy[1] -= k * y[1]
    return dy

In [5]:
times = np.linspace(0, 10, 1000)
m = 1
M = 2
g = 9.81
l = 1
# w = 1 # 5.424942396007538
w = 20

Y0 = [deg2rad(.1), 0.]


# Y0 = [np.pi*0.95, 0.]
print(f'w0 = {np.sqrt(g/l * (m+M)/m):.2f}')

w0 = 5.42


**First case:** $\cos\phi_0 = \frac{g}{l\omega^2}\frac{m+M}{m}$

In [6]:
k = 0
sol = solve_ivp(derivative, [times[0], times[-1]], Y0, t_eval=times, 
                method='DOP853',
#                 method='LSODA',
#                 method='RK23', 
#                 rtol=1e-8,
#                 atol=1e-10
               )
k = 1
sol2 = solve_ivp(derivative, [times[0], times[-1]], Y0, t_eval=times, method='DOP853', rtol=1e-8, atol=1e-10)


fig, ax = figure('example', 1, 2, figsize=(12, 4))

ax[0].plot(sol.t, sol.y[0]/np.pi, label='phi')
ax[0].plot(sol.t, sol2.y[0]/np.pi, 'k--', label='phi0')

# ax[0].plot(sol.t, sol.y[1], label='p')
steady = g/(l*w**2) * (m+M)/m
if steady <= 1:
    ax[0].plot(sol.t, [np.arccos(steady)/np.pi]*len(times), 'k-')
ax[1].plot(sol.t, energy(sol.y)[0], label='Epot')
ax[1].plot(sol.t, energy(sol.y)[1], label='Ekin')

ax[0].legend()
ax[1].legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**Second case:** $\cos\phi_0 = 0$

In [7]:
w = 5.
k = 0
times = np.linspace(0, 5, 100)
sol3 = solve_ivp(derivative, [times[0], times[-1]], Y0, t_eval=times, method='DOP853', rtol=1e-8, atol=1e-10)
w = 10
times = np.linspace(0, 0.75, 100)
sol4 = solve_ivp(derivative, [times[0], times[-1]], Y0, t_eval=times, method='DOP853', rtol=1e-8, atol=1e-10)

fig, ax = figure('small', 1, 2, figsize=(12, 4))

ax[0].plot(sol3.t, sol3.y[0]/np.pi, label='phi')
w = 5.
w2 = np.sqrt(g/l * (m+M)/m - w**2)
ax[0].plot(sol3.t, Y0[0]/np.pi * np.cos(w2*sol3.t), 'k--', label='analytic')

ax[1].plot(sol4.t, sol4.y[0]/np.pi, label='phi')
w = 10
w2 = np.sqrt(w**2 - g/l * (m+M)/m)
ax[1].plot(sol4.t, Y0[0]/np.pi * np.cosh(w2*sol4.t), 'k--')


ax[0].legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**Oscillation around $\theta_0$**

In [8]:
w = 10
k = 0
times = np.linspace(0, 4, 100)
theta = np.arccos(g/(l*w**2) * (m+M)/m)
Y0 = [theta+ 0.01, 0]
sol5 = solve_ivp(derivative, [times[0], times[-1]], Y0, t_eval=times, method='DOP853', rtol=1e-10, atol=1e-12)


fig, ax = figure('t0', 1, 1, figsize=(6, 4))

ax.plot(sol5.t, sol5.y[0]/np.pi, label='phi')
w2 = np.sqrt(m*w**2*sin(theta)**2/(m+2*M*sin(theta)**2))
ax.plot(sol5.t, 0.01/np.pi * np.cos(w2*sol5.t) + theta/np.pi, 'k--', label='analytic')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Parameters

In [9]:
fig, ax = figure('para', 1, 2, figsize=(12, 4))
N=1000
ws = np.linspace(0, 20, N)
ind = ws > np.sqrt(g/l*(m+M)/m)
ax[0].plot(ws, [0]*N, 'k--')
ax[0].plot(ws[~ind], [0]*len(ws[~ind]), 'k')
ax[0].plot(ws[ind], np.arccos(g/(l*ws[ind]**2)*(m+M)/m)/np.pi, 'b')
ax[0].plot(ws[ind], -np.arccos(g/(l*ws[ind]**2)*(m+M)/m)/np.pi, 'r')
ax[0].set_xlabel('$\omega$')
ax[0].set_ylabel('$\\theta_0/\pi$')

ws = np.linspace(0, 5.42, N)
w2 = np.sqrt(g/l * (M+m)/m - ws**2)
ax[1].plot(ws, w2, 'k')
ax[1].set_xlabel('$\omega$')
ax[1].set_ylabel('$\\omega_0$')

ws = np.linspace(5.43, 20, N)
theta = np.arccos(g/(l*ws**2) * (m+M)/m)
w2 = np.sqrt(m*ws**2*sin(theta)**2/(m+2*M*sin(theta)**2))
ax[1].plot(ws, w2, 'b')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …